In [49]:
from datetime import datetime, date, timedelta
import variables
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import yfinance as yf
import pandas as pd 
import sys

In [50]:
# get Path to csv
PATH = variables.stocks

In [51]:
# Get Dataframes from csv
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + ".csv", index_col=0)
    except FileNotFoundError:
        print("File is not here" )
        print('Expected file: ' + PATH + ticker + ".csv")
    else: 
        return df


In [52]:
# Get all stock tickers
files = [x for x in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
# tickers.remove('.ds_Store')
tickers.sort()
len(tickers)

3261

In [54]:
old_df = get_stock_df_from_csv("A")
old_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging,cum_return
Date,,,,,,,,,,,,,,,,,
2017-02-08,0.262611,0.271365,0.262611,0.271365,11550,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288872,NaN
2017-02-09,0.262611,0.271365,0.262611,0.271365,46970,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288872,1.000000
2017-02-10,0.271365,0.280119,0.271365,0.271365,76010,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.280118,1.000000
2017-02-13,0.271365,0.280118,0.271365,0.280118,2420,0.0,0.0,0.032258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.280118,1.032258
2017-02-14,0.280118,0.280118,0.280118,0.280118,1100,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.271365,1.032258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-02,8.000000,8.060000,7.270000,7.310000,254800,0.0,0.0,-0.074684,8.1810,9.249134,7.112866,7.720,8.3,10.164561,11.221028,NaN,26.937916
2022-02-03,7.210000,7.550000,7.130000,7.290000,284800,0.0,0.0,-0.002736,8.0805,9.084197,7.076803,7.595,8.3,10.112861,11.221028,NaN,26.864214
2022-02-04,7.290000,7.620000,7.220000,7.520000,154600,0.0,0.0,0.031550,8.0125,8.973224,7.051776,7.595,8.3,10.018454,11.221028,NaN,27.711782


In [42]:
start_date = (datetime.strptime(old_df.index[-1], "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
end_date = datetime.today().strftime("%Y-%m-%d")

In [55]:
new_df = yf.download("A", start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'), index=['x', 'y'])
df
   A  B
x  1  2
y  3  4
df2 = pd.DataFrame([[5, 6], [7, 8]], columns=list('AB'), index=['x', 'y'])
df.append(df2)

In [56]:
new_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-09,142.729996,145.100006,142.149994,144.440002,144.440002,1216900
2022-02-10,141.559998,144.039993,140.580002,141.660004,141.660004,1063700
2022-02-11,141.119995,142.509995,136.240005,137.199997,137.199997,1321900
2022-02-14,136.960007,137.690002,134.100006,135.210007,135.210007,2205300


In [57]:
final_df = old_df.append(new_df)
final_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging,cum_return,Adj Close
Date,,,,,,,,,,,,,,,,,,
2017-02-08,0.262611,0.271365,0.262611,0.271365,11550,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288872,NaN,NaN
2017-02-09,0.262611,0.271365,0.262611,0.271365,46970,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288872,1.000000,NaN
2017-02-10,0.271365,0.280119,0.271365,0.271365,76010,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.280118,1.000000,NaN
2017-02-13,0.271365,0.280118,0.271365,0.280118,2420,0.0,0.0,0.032258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.280118,1.032258,NaN
2017-02-14,0.280118,0.280118,0.280118,0.280118,1100,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.271365,1.032258,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-08,7.410000,7.550000,7.340000,7.450000,32663,0.0,0.0,0.005398,7.8915,8.802676,6.980324,7.53,8.3,9.845373,11.221028,NaN,27.453826,NaN
2022-02-09 00:00:00,142.729996,145.100006,142.149994,144.440002,1216900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.440002
2022-02-10 00:00:00,141.559998,144.039993,140.580002,141.660004,1063700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.660004


In [58]:
import columns_modules as cm

df = cm.add_daily_return_to_df(final_df)

df 

,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,middle_band,upper_band,lower_band,Conversion,Baseline,SpanA,SpanB,Lagging,cum_return,Adj Close
Date,,,,,,,,,,,,,,,,,,
2017-02-08,0.262611,0.271365,0.262611,0.271365,11550,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288872,NaN,NaN
2017-02-09,0.262611,0.271365,0.262611,0.271365,46970,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288872,1.000000,NaN
2017-02-10,0.271365,0.280119,0.271365,0.271365,76010,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.280118,1.000000,NaN
2017-02-13,0.271365,0.280118,0.271365,0.280118,2420,0.0,0.0,0.032258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.280118,1.032258,NaN
2017-02-14,0.280118,0.280118,0.280118,0.280118,1100,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.271365,1.032258,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-08,7.410000,7.550000,7.340000,7.450000,32663,0.0,0.0,0.005398,7.8915,8.802676,6.980324,7.53,8.3,9.845373,11.221028,NaN,27.453826,NaN
2022-02-09 00:00:00,142.729996,145.100006,142.149994,144.440002,1216900,NaN,NaN,18.387920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.440002
2022-02-10 00:00:00,141.559998,144.039993,140.580002,141.660004,1063700,NaN,NaN,-0.019247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.660004


In [35]:
df